In [26]:
    # Генерируем ответ на основе сообщений
# Используем pipeline как высокоуровневый помощник
from transformers import pipeline

# Используем модель Phi-3-mini-4k-instruct
pipe = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

import json
data = json.load(open('data.json', 'r', encoding='utf-8'))
for el in data:
    messages = [{"role": "user", "content": f'!!!Напиши только Yes/No!!! скажи есть ли решение этого вопроса {"Какие требования к программному обеспечению для работы с порталом?"} в этой статье {data[el]} '}]
    response = pipe(messages, max_new_tokens=512, do_sample=True, temperature=0.7, use_cache=False)
    print(el, response[0]['generated_text'][-1]['content'])

IndentationError: expected an indented block after 'for' statement on line 10 (2532313086.py, line 11)

In [10]:
import pandas as pd
df = pd.read_parquet('docs/dataset.parquet')
data = dict()
for i in range(len(df)):
    data[i] = [df.iloc[i]['Заголовок статьи'], df.iloc[i]['Описание']]

import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)


In [ ]:
# Импортируем библиотеку SymSpell для проверки орфографии
from symspellpy import SymSpell, Verbosity
import os
import pandas as pd

# Инициализируем SymSpell с правильными параметрами
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Загружаем словарь для русского языка
# SymSpell использует словарь частотности слов для работы
dictionary_path = "dictionary_ru.txt"

# Загружаем словарь в SymSpell
if not sym_spell.load_dictionary(dictionary_path, encoding='utf-8', term_index=0, count_index=1):
    print(f"Ошибка загрузки словаря из {dictionary_path}")

# Функция для проверки орфографии
def check_spelling(word):
    # Получаем предложения для исправления
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    
    if suggestions:
        return f"Возможно, вы имели в виду: {suggestions[0].term} (расстояние: {suggestions[0].distance})"
    return "Исправлений не найдено. Возможно, префикс слишком короткий или словарь не содержит подходящих слов."

# Проверяем ввод пользователя
user_input = "звявка"
print(check_spelling(user_input))


In [ ]:
import requests

# URL для получения списка всех документов
url = "http://localhost:7777/query" 

# Параметры запроса
params = {
    "query": "звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки "
}

# Выполняем GET-запрос
response = requests.get(url, params=params)

# Выводим ответ
print(response.json())  


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask



#Sentences we want sentence embeddings for
sentences = ['Привет! Как твои дела?',
             'А правда, что 42 твое любимое число?']

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
